<h1>IBM Capstone Project</h1>

<h2>Week 5 Final Project</h2>

<h3>Opening a New Shopping Mall in Oslo, Norway</h3>

<ul>
    <li>Build a dataframe of neighbourshoods in Delhi,India by webscrapping from Wikipedia page.</li>
    <li>Get coordinates of the neighbourhoods.</li>
    <li>Using Foursquare API to obtain Shopping Mall Data.</li>
    <li>Cluster the neighbourhoods.</li>
    <li>Selection of best cluster to open a new shoping mall.</li>
</ul>

<h2>Installing and Importing Required Libraries</h2>

In [4]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 94 kB 3.6 MB/s  eta 0:00:01


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<h2>Scrapping data from Wikipedia Page Using BeautifulSoup</h2>

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Oslo").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Abildsø
1,Adamstuen
2,Aker Brygge
3,Alnabru
4,Ammerud


In [11]:
kl_df = kl_df.iloc[1:,]


In [12]:
# print the number of rows of the dataframe
kl_df.head()

,Neighborhood
1,Adamstuen
2,Aker Brygge
3,Alnabru
4,Ammerud
5,Årvoll


<h2>Getting coordinates Of Neighbourhoods</h2>

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Oslo, Norway'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [16]:
coords

[[59.93282000000005, 10.734520000000032],
 [59.91026000000005, 10.727190000000064],
 [59.92677610422236, 10.835322790373747],
 [59.95869000000005, 10.870730000000037],
 [59.946240000000046, 10.814580000000035],
 [59.941214585863854, 10.784105897498437],
 [59.883890000000065, 10.777500000000032],
 [60.02666000000005, 10.583450000000028],
 [59.9215100286163, 10.654530073820865],
 [59.9250410428498, 10.731553429961028],
 [59.940120000000036, 10.82289000000003],
 [59.94362000000007, 10.759820000000047],
 [59.837870000000066, 10.829030000000046],
 [61.36211000000003, 5.578380000000038],
 [59.93743000000006, 10.71819000000005],
 [59.87535000000008, 10.841810000000066],
 [59.88409000000007, 10.84489000000002],
 [59.92728358533747, 10.73541538438898],
 [59.93480000000005, 10.695120000000031],
 [59.94747836289172, 10.859501311487826],
 [59.92091439174288, 10.716800847673346],
 [59.91047480722336, 10.810626056652737],
 [59.900000000000034, 10.683330000000069],
 [59.942875905299545, 10.7996738982

In [17]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [18]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [19]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(117, 3)


,Neighborhood,Latitude,Longitude
1,Adamstuen,59.910260,10.727190
2,Aker Brygge,59.926776,10.835323
3,Alnabru,59.958690,10.870730
4,Ammerud,59.946240,10.814580
5,Årvoll,59.941215,10.784106
6,"Åsen, Oslo",59.883890,10.777500
7,Bekkelaget,60.026660,10.583450
8,"Berg, Oslo",59.921510,10.654530
9,Bestum,59.925041,10.731553
10,Bislett,59.940120,10.822890


<h2>Replacing Nan values Manually
</h2>

In [21]:
kl_df['Latitude'] = kl_df['Latitude'].replace(np.nan, 59.966670)
kl_df['Longitude'] = kl_df['Longitude'].replace(np.nan, 10.666671)

In [23]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

<h2>Creating a map of Oslo with Neighbourhoods on Top</h2>

In [24]:
# get the coordinates of Oslo
address = 'Oslo, Norway'

geolocator = Nominatim(user_agent="locdata")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oslo, Norway {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oslo, Norway 59.9133301, 10.7389701.


In [25]:
# create map of Delhi using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

<h2>Using the Foursquare API to explore the Neighbourhoods</h2>

In [26]:
# define Foursquare Credentials and Version
CLIENT_ID = 'JXRFDLSKT32TJ3SRTO1MKL024XSJR3H4OECLJ4NE5LUAJKZU' # your Foursquare ID
CLIENT_SECRET = 'JAR3VABNSAGBMRLX4Y1N2NQMIX2YQRSFQI5JJR0PG2SMMYLS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JXRFDLSKT32TJ3SRTO1MKL024XSJR3H4OECLJ4NE5LUAJKZU
CLIENT_SECRET:JAR3VABNSAGBMRLX4Y1N2NQMIX2YQRSFQI5JJR0PG2SMMYLS


<h2>Getting top 100 venues within a radius of 2000 meters
</h2>

In [32]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [33]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7033, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adamstuen,59.91026,10.72719,Vinmonopolet (Aker Brygge),59.910870,10.725606,Wine Shop
1,Adamstuen,59.91026,10.72719,Delicatessen,59.910807,10.727133,Tapas Restaurant
2,Adamstuen,59.91026,10.72719,Paradis Gelateria,59.908192,10.723471,Ice Cream Shop
3,Adamstuen,59.91026,10.72719,Døgnvill Burger Tjuvholmen,59.908258,10.723764,American Restaurant
4,Adamstuen,59.91026,10.72719,Burger Joint,59.910727,10.726730,Burger Joint


<h2>Checking the returned venues from each neighbourhood
</h2>

In [34]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adamstuen,100,100,100,100,100,100
Aker Brygge,54,54,54,54,54,54
Alnabru,16,16,16,16,16,16
Ammerud,32,32,32,32,32,32
Bekkelaget,4,4,4,4,4,4
"Berg, Oslo",49,49,49,49,49,49
Bestum,100,100,100,100,100,100
Bislett,38,38,38,38,38,38
Bjerkebanen,100,100,100,100,100,100


<h3>Finding number of Unique Venues
</h3>

In [36]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 215 uniques categories.


In [39]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Wine Shop', 'Tapas Restaurant', 'Ice Cream Shop',
       'American Restaurant', 'Burger Joint', 'Hotel', 'Movie Theater',
       'Indian Restaurant', 'Italian Restaurant',
       'South American Restaurant', 'Art Gallery',
       'Scandinavian Restaurant', 'Chinese Restaurant',
       'Sushi Restaurant', 'Castle', 'Sandwich Place', 'Pizza Place',
       'Toy / Game Store', 'Coffee Shop', 'Café', 'Theater', 'Waterfront',
       'Massage Studio', 'Park', 'Steakhouse', 'Japanese Restaurant',
       'Seafood Restaurant', 'Vegetarian / Vegan Restaurant', 'Bakery',
       'Jazz Club', 'Clothing Store', 'Bookstore', 'Bar', 'Museum',
       'Plaza', 'Whisky Bar', 'Palace', 'Dim Sum Restaurant',
       'Restaurant', 'Spa', 'Camera Store', 'Pub', 'French Restaurant',
       'Salad Place', 'Fountain', 'Record Shop', 'Performing Arts Venue',
       'Music Venue', 'Asian Restaurant', 'Gym / Fitness Center',
       'History Museum', 'Other Nightlife', 'Beer Bar',
       'Sporting Goods Shop'

<h2>One hot encoding Neighbourhoods
</h2>

In [40]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7033, 216)


,Neighborhoods,Advertising Agency,American Restaurant,Amphitheater,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Castle,Cemetery,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dive Bar,Dog Run,Dumpling Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Newsstand,Nightclub,Nudist Beach,Opera House,Other Great Outdoors,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Ski Trail,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Adamstuen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Adamstuen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adamstuen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

<h2>Using mean to find frequency of occurence in each category
</h2>

In [42]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(116, 216)


,Neighborhoods,Advertising Agency,American Restaurant,Amphitheater,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Castle,Cemetery,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dive Bar,Dog Run,Dumpling Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Newsstand,Nightclub,Nudist Beach,Opera House,Other Great Outdoors,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Ski Trail,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Adamstuen,0.000000,0.01,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.040000,0.010000,0.0000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.010000,0.000000,0.00,0.02,0.000000,0.010000,0.000000,0.030000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.010000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.01,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.000000,0.000,0.020000,0.070000,0.00,0.000000,0.020000,0.030000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000

<h2>Grouping and Creating New DataFrame for Shoping Mall Data Only</h2>

In [45]:
len(kl_grouped[kl_grouped["Community Center"] > 0])


4

In [48]:
kl_languageC = kl_grouped[["Neighborhoods","Community Center"]]


In [51]:
kl_languageC.head()


,Neighborhoods,Community Center
0,Adamstuen,0.0
1,Aker Brygge,0.0
2,Alnabru,0.0
3,Ammerud,0.0
4,Bekkelaget,0.0


<h2>Clusturing Neighbourhoods
</h2>

In [55]:
# set number of clusters
kclusters = 3

kl_clustering = kl_languageC.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [57]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_languageC.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [58]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Community Center,Cluster Labels
0,Adamstuen,0.0,0
1,Aker Brygge,0.0,0
2,Alnabru,0.0,0
3,Ammerud,0.0,0
4,Bekkelaget,0.0,0


In [59]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(116, 5)


,Neighborhood,Community Center,Cluster Labels,Latitude,Longitude
0,Adamstuen,0.0,0,59.910260,10.727190
1,Aker Brygge,0.0,0,59.926776,10.835323
2,Alnabru,0.0,0,59.958690,10.870730
3,Ammerud,0.0,0,59.946240,10.814580
4,Bekkelaget,0.0,0,60.026660,10.583450


In [60]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(116, 5)


,Neighborhood,Community Center,Cluster Labels,Latitude,Longitude
0,Adamstuen,0.000000,0,59.910260,10.727190
82,Rødtvet,0.000000,0,59.966780,10.898180
81,Røa,0.000000,0,59.925180,10.770530
80,Ryen,0.000000,0,59.898330,10.793060
79,Rosenhoff,0.000000,0,59.895180,10.805620
78,Romsås,0.000000,0,59.929554,10.774535
77,Rodeløkka,0.000000,0,59.951350,10.858280
76,"Ris, Norway",0.000000,0,59.946500,10.643780
75,Pipervika,0.000000,0,59.913740,10.740910
74,Pilestredet Park,0.000000,0,59.916670,10.733330


<h2>Using Follium to Visualize Clusters
</h2>

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examining Clusters for Observation
</h2>

<h4>Cluster 0</h4>

In [63]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Community Center,Cluster Labels,Latitude,Longitude
0,Adamstuen,0.0,0,59.910260,10.727190
82,Rødtvet,0.0,0,59.966780,10.898180
81,Røa,0.0,0,59.925180,10.770530
80,Ryen,0.0,0,59.898330,10.793060
79,Rosenhoff,0.0,0,59.895180,10.805620
78,Romsås,0.0,0,59.929554,10.774535
77,Rodeløkka,0.0,0,59.951350,10.858280
76,"Ris, Norway",0.0,0,59.946500,10.643780
75,Pipervika,0.0,0,59.913740,10.740910
74,Pilestredet Park,0.0,0,59.916670,10.733330


<h4>Cluster 1</h4>

In [64]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]


,Neighborhood,Community Center,Cluster Labels,Latitude,Longitude
92,Sørengautstikkeren,0.010753,1,59.945060,10.777720
113,Årvoll,0.010000,1,59.941215,10.784106


<h4>Cluster 2</h4>

In [66]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]


,Neighborhood,Community Center,Cluster Labels,Latitude,Longitude
19,Bygdøy,0.011905,2,59.942876,10.799674
33,"Gimle, Oslo",0.011494,2,59.947960,10.784030


<h5>
Observations:
Most of the shopping malls are concentrated in the outer area of Delhi city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.
</h5>